In [1]:
from pathlib import Path
import json
import re

import torch
from torch import Tensor
import torch.optim as optim
from torch.utils.data import Dataset
from torch import nn
from torch import functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from matplotlib import image as mpl_image
from typing import Type, Any, Callable, Union, List, Optional

from hmmlearn import hmm


In [2]:
batch_size = 4

classes = {"Not_Oriented": 0, "Oriented": 1, "Precieved_Not_Oriented": 2, "Precieved_Oriented": 3, "Slanted": 4, "Unidentified": 5}

In [3]:
test_folder = Path("/nic/data/high_low/train")
model_file = Path("/nic/models/model_resnet_18_high_low_affine_149.pyt")
annotations_file = Path("/nic/annotations.json")
sequences_file = Path("/nic/sequences.npy")
emission_matrix_path = Path("/nic/emission_matrix.npy")
emission_matrix_path_three_classes = Path("/nic/emission_matrix_three_classes.npy")

n_iter=1000

In [4]:
[x for x in test_folder.rglob("*.png")]

[WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[0].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[10].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[14].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[15].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[16].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[18].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[19].png'),
 WindowsPath('/nic/data/high_low/train/Not Oriented/video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[1].png'),
 WindowsPath('/nic/data/hi

In [5]:
class ImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = path.parts[-2]
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"][:,:,:3])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [6]:
class BlindImageDataset(Dataset):
    def __init__(self, dataset_folders_path, image_transform, target_transform):
        
        self.dataset_folders_path = dataset_folders_path
        
        self.data = {}
        for path in dataset_folders_path.rglob("*.png"):
            label = "Unidentified"
            image = mpl_image.imread(str(path))#torchvision.read_image(path)
            self.data[str(path)] = {"image": image, "label": label}
            
        self.keys = list(self.data.keys())
        
        self.image_transform = image_transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        sample_key = self.keys[idx]
        sample = self.data[sample_key]
        image = self.image_transform(sample["image"][:,:,:3])
        label = self.target_transform(sample["label"])
#         print(sample)
        
        return {"image": image, "label": label, "path": str(sample_key)}

In [7]:
def label_to_tensor(label, classes):
    index = classes[label]

    return index

In [8]:


image_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        torchvision.transforms.Grayscale(num_output_channels=1),
        torchvision.transforms.Normalize(0.0, 1.0, inplace=False),
        torchvision.transforms.RandomAffine(
            (-180.0, 180.0), 
            translate=(0.25,0.25),
            interpolation=transforms.functional.InterpolationMode.BILINEAR,
        ),
        torchvision.transforms.GaussianBlur(
            (3,3), 
            sigma=(0.01, 2.0),
        ),
    ]
)



target_transform = lambda x: label_to_tensor(x, classes)


In [9]:
testset = BlindImageDataset(test_folder, image_transform, target_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, 
                                        # num_workers=1
                                         drop_last=True
                                        )

# Define the model

In [10]:

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 6,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
#         print(x.shape)
        x = self.layer2(x)
#         print(x.shape)
        x = self.layer3(x)
#         print(x.shape)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [11]:
def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [12]:
def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

# Load the model

In [13]:
model = resnet18()


In [14]:
model.load_state_dict(torch.load(str(model_file), map_location=torch.device("cpu")))


<All keys matched successfully>

# Annotate the data with the model

In [15]:
confusion_matrix_test = np.zeros((len(classes), len(classes)))
annotations = {}
for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
    path = data["path"]
    inputs = data["image"]
    labels = data["label"]
    
    
    # forward + backward + optimize
    outputs = model(inputs)


    
    for j in [0,1,2,3]:
        confusion_matrix_test[labels[j], torch.argmax(outputs[j])] += 1
        label = int(torch.argmax(outputs[j]).detach().cpu())
        annotations[path[j]] = label
        

# Output the confusion matrix

In [16]:
confusion_matrix_test_table = pd.DataFrame(
data=confusion_matrix_test,
index=list(classes.keys()),
columns=list(classes.keys()))

In [17]:
confusion_matrix_test_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified
Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Not_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Precieved_Oriented,0.0,0.0,0.0,0.0,0.0,0.0
Slanted,0.0,0.0,0.0,0.0,0.0,0.0
Unidentified,2460.0,4360.0,5657.0,6944.0,5515.0,180.0


# Save to json

In [18]:
annotations

{'\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[0].png': 0,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[10].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[14].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[15].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[16].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[18].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[19].png': 4,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[1].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_21

In [19]:
new_annotations = {}

for path, annotation in annotations.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_annotations[particle][frame] = annotation
    except:
        new_annotations[particle] = {}
        new_annotations[particle][frame] = annotation

In [20]:
sequences = {}
ordered_annotations = {}

k = 0
sequence = []
for particle in new_annotations:
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_annotations[particle][j]
            sequence.append(next_annotation)
        except:
            sequence.append(6)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [21]:
sequences

{0: [2, 2, 2, 4, 4, 4, 4, 4, 3, 2, 4, 2, 3, 2, 2, 4, 4, 5, 2, 3],
 1: [3, 4, 2, 4, 3, 4, 3, 1, 1, 2, 3, 3, 4, 1, 1, 1, 4, 2, 4, 2],
 2: [3, 4, 3, 3, 3, 4, 4, 4, 2, 3, 4, 4, 3, 4, 3, 2, 4, 3, 4, 3],
 3: [4, 2, 3, 4, 3, 2, 4, 4, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 2, 4],
 4: [4, 2, 3, 3, 2, 4, 3, 2, 4, 4, 4, 1, 3, 3, 4, 4, 3, 4, 4, 3],
 5: [4, 3, 4, 4, 2, 4, 3, 4, 3, 3, 4, 4, 3, 2, 4, 4, 4, 4, 4, 5],
 6: [5, 5, 3, 3, 1, 1, 4, 3, 3, 3, 4, 5, 2, 2, 2, 2, 3, 1, 3, 3],
 7: [2, 4, 2, 4, 4, 4, 3, 3, 1, 3, 4, 4, 2, 2, 4, 3, 4, 4, 4, 2],
 8: [3, 3, 4, 3, 3, 3, 3, 3, 4, 4, 4, 2, 3, 3, 4, 4, 4, 3, 4, 4],
 9: [4, 2, 4, 3, 4, 3, 4, 3, 4, 4, 4, 4, 3, 4, 4, 4, 3, 3, 1, 3],
 10: [4, 4, 0, 4, 4, 2, 4, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4],
 11: [4, 2, 3, 3, 3, 2, 2, 4, 4, 2, 2, 2, 2, 0, 2, 2, 2, 1, 1, 5],
 12: [3, 0, 0, 4, 3, 2, 4, 4, 2, 5, 0, 2, 2, 2, 2, 2, 5, 4, 2, 5],
 13: [2, 2, 2, 4, 2, 4, 4, 4, 4, 4, 3, 4, 5, 5, 2, 2, 2, 2, 2, 0],
 14: [3, 3, 4, 0, 2, 2, 2, 3, 4, 4, 4, 2, 2, 3, 4, 4, 3, 3, 4, 4],
 15: 

In [22]:
full_sequences = [sequence for sequence in sequences.values() if len(sequence) == 20]

In [23]:
sequences_array = np.array(list(full_sequences))
sequences_array.shape

(269, 20)

In [24]:
lengths = [sequences_array.shape[1]]*sequences_array.shape[0]

In [25]:
np.save(str(sequences_file), sequences_array)

In [26]:
naive_transition_matrix = np.zeros((7,7))
for sequence in sequences.values():
    prev_state = 6
    for state in sequence:
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state


In [27]:

classes = {
    "Not_Oriented": 0, 
    "Oriented": 1, 
    "Precieved_Not_Oriented": 2, 
    "Precieved_Oriented": 3, 
    "Slanted": 4, 
    "Unidentified": 5, 
    "No_sample": 6,
}
naive_transition_matrix = naive_transition_matrix / np.sum(naive_transition_matrix, axis=1).reshape((-1,1))
print(np.sum(naive_transition_matrix, axis=1))

naive_transition_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
 

[1. 1. 1. 1. 1. 1. 1.]


## Output the naive transition matrix

In [28]:
naive_transition_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,0.329114,0.016878,0.502110,0.025316,0.122363,0.004219,0.000000
Oriented,0.002740,0.320548,0.076712,0.427397,0.161644,0.010959,0.000000
Precieved_Not_Oriented,0.093750,0.021780,0.390152,0.144886,0.317235,0.027462,0.004735
Precieved_Oriented,0.009181,0.099576,0.093220,0.427260,0.363701,0.006356,0.000706
Slanted,0.011936,0.031136,0.169694,0.265698,0.499222,0.022314,0.000000
Unidentified,0.010204,0.020408,0.306122,0.204082,0.295918,0.163265,0.000000
No_sample,0.092527,0.096085,0.185053,0.224199,0.334520,0.028470,0.039146


# Hidden Markov model

## Load the emission matrix

In [29]:
emission_matrix_np = np.load(emission_matrix_path)

In [30]:
emission_matrix_np

array([[6.43678161e-01, 0.00000000e+00, 3.39901478e-01, 1.64203612e-03,
        1.31362890e-02, 1.64203612e-03, 0.00000000e+00],
       [0.00000000e+00, 8.45682451e-01, 5.57103064e-04, 1.50417827e-01,
        2.22841226e-03, 1.11420613e-03, 0.00000000e+00],
       [2.91530945e-01, 0.00000000e+00, 4.62540717e-01, 5.70032573e-03,
        2.39413681e-01, 8.14332248e-04, 0.00000000e+00],
       [0.00000000e+00, 4.34234234e-01, 0.00000000e+00, 5.19819820e-01,
        4.59459459e-02, 0.00000000e+00, 0.00000000e+00],
       [1.41998908e-02, 8.19224468e-03, 1.60567996e-01, 2.43036592e-01,
        5.74003277e-01, 0.00000000e+00, 0.00000000e+00],
       [3.54430380e-02, 2.78481013e-02, 1.82278481e-01, 2.86075949e-01,
        3.36708861e-01, 1.31645570e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [31]:
emission_matrix_np.shape

(7, 7)

## Define the model

In [32]:
model = hmm.MultinomialHMM(n_components=7, n_iter=n_iter, params="st", init_params="st")

In [33]:
model.emissionprob_ = emission_matrix_np

In [34]:
sequences_array.shape

(269, 20)

## Fit the model

In [35]:
model.fit(sequences_array.reshape(-1, 1), lengths)

MultinomialHMM(init_params='st', n_components=7, n_iter=1000, params='st',
               random_state=RandomState(MT19937) at 0x21AED1BE048)

## Format the model output

In [36]:
classes = {
    "Not_Oriented": 0, 
    "Oriented": 1, 
    "Precieved_Not_Oriented": 2, 
    "Precieved_Oriented": 3, 
    "Slanted": 4, 
    "Unidentified": 5, 
    "No_sample": 6,
}
transition_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))


## Output the transition matrix

In [37]:
transition_table

,Not_Oriented,Oriented,Precieved_Not_Oriented,Precieved_Oriented,Slanted,Unidentified,No_sample
Not_Oriented,8.489245e-01,5.457615e-103,1.510755e-01,4.007009e-68,3.538721e-11,5.266096e-17,7.706415e-242
Oriented,6.445540e-36,8.325385e-01,2.241513e-02,1.450288e-01,1.421102e-11,1.754941e-05,0.000000e+00
Precieved_Not_Oriented,1.392700e-13,3.157496e-17,8.845026e-01,1.289653e-03,1.352891e-02,9.155469e-02,9.124108e-03
Precieved_Oriented,4.836323e-13,3.870289e-18,3.582112e-03,8.451044e-01,1.326371e-01,1.735870e-02,1.317704e-03
Slanted,1.064527e-37,8.389937e-26,2.270830e-12,2.874218e-02,9.218155e-01,4.944232e-02,2.640986e-50
Unidentified,2.730901e-22,6.357580e-27,4.916587e-02,4.498121e-02,1.901893e-01,7.156636e-01,2.166236e-47
No_sample,8.838939e-18,0.000000e+00,8.333333e-02,0.000000e+00,2.232072e-74,2.005765e-65,9.166667e-01


# Hidden Markov model: Three Classes

In [38]:
annotations

{'\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[0].png': 0,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[10].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[14].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[15].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[16].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[18].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[19].png': 4,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_210219_NGasl_03_20MLE.vsi.tif]_particle[0]_frame[1].png': 2,
 '\\nic\\data\\high_low\\train\\Not Oriented\\video[MAX_21

In [39]:
annotation_mapping = {
    0: 0, 
    1: 1, 
    2: 0, 
    3: 1, 
    4: 2, 
    5: 3, 
    6: 4,
}

In [40]:
new_annotations = {}

for path, annotation in annotations.items():
    regex = "_particle\[([0-9]+)\]_frame\[([0-9]+)\]"
    capture = re.findall(regex, path)
    particle = int(capture[0][0])
    frame = int(capture[0][1])
    try:
        new_annotations[particle][frame] = annotation_mapping[annotation]
    except:
        new_annotations[particle] = {}
        new_annotations[particle][frame] = annotation_mapping[annotation]

In [41]:
sequences = {}
ordered_annotations = {}

k = 0
sequence = []
for particle in new_annotations:
    ordered_annotations[particle] = []
    
    for j in range(20):
        try:
            next_annotation = new_annotations[particle][j]
            sequence.append(next_annotation)
        except:
            sequence.append(4)
    sequences[k] = sequence
    k = k + 1
    sequence = []
    

In [42]:
full_sequences = [sequence for sequence in sequences.values() if len(sequence) == 20]

In [43]:
full_sequences

[[0, 0, 0, 2, 2, 2, 2, 2, 1, 0, 2, 0, 1, 0, 0, 2, 2, 3, 0, 1],
 [1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 0, 2, 0],
 [1, 2, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 0, 2, 1, 2, 1],
 [2, 0, 1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2],
 [2, 0, 1, 1, 0, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1],
 [2, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 2, 1, 0, 2, 2, 2, 2, 2, 3],
 [3, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 3, 0, 0, 0, 0, 1, 1, 1, 1],
 [0, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 2, 1, 2, 2, 2, 0],
 [1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2],
 [2, 0, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 1],
 [2, 2, 0, 2, 2, 0, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2],
 [2, 0, 1, 1, 1, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 3],
 [1, 0, 0, 2, 1, 0, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0, 3, 2, 0, 3],
 [0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 3, 3, 0, 0, 0, 0, 0, 0],
 [1, 1, 2, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 1, 2, 2, 1, 1, 2, 2],
 [2, 2, 2, 3, 1, 2, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 1, 1,

In [44]:
sequences_array = np.array(list(full_sequences))
sequences_array.shape

(269, 20)

In [45]:
lengths = [sequences_array.shape[1]]*sequences_array.shape[0]

In [46]:
np.save(str(sequences_file), sequences_array)

In [47]:
naive_transition_matrix = np.zeros((5,5))
for sequence in sequences.values():
    prev_state = 4
    for state in sequence:
        naive_transition_matrix[prev_state, state] += 1
        prev_state = state


In [48]:

classes = {
    "Not_Oriented": 0, 
    "Oriented": 1, 
    "Slanted": 2, 
    "Unidentified": 3, 
    "No_sample": 4,
}
naive_transition_matrix = naive_transition_matrix / np.sum(naive_transition_matrix, axis=1).reshape((-1,1))
print(np.sum(naive_transition_matrix, axis=1))

naive_transition_table = pd.DataFrame(
data=naive_transition_matrix,
index=list(classes.keys()),
columns=list(classes.keys()))
 

[1. 1. 1. 1. 1.]


## Load the emission matrix

In [49]:
emission_matrix_np = np.load(emission_matrix_path_three_classes)

In [50]:
emission_matrix_np

array([[8.52476864e-01, 2.72182907e-03, 1.43168209e-01, 1.63309744e-03,
        0.00000000e+00],
       [1.03270224e-03, 9.76936317e-01, 2.13425129e-02, 6.88468158e-04,
        0.00000000e+00],
       [1.97160022e-01, 2.29382851e-01, 5.72910978e-01, 5.46149645e-04,
        0.00000000e+00],
       [2.91139241e-01, 3.18987342e-01, 2.48101266e-01, 1.41772152e-01,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00]])

## Define the model

In [51]:
model = hmm.MultinomialHMM(n_components=5, n_iter=n_iter, params="st", init_params="st")

In [52]:
model.emissionprob_ = emission_matrix_np

## Fit the model

In [53]:
model.fit(sequences_array.reshape(-1, 1), lengths)

MultinomialHMM(init_params='st', n_components=5, n_iter=1000, params='st',
               random_state=RandomState(MT19937) at 0x21AED1BE048)

## Format the model output

In [58]:
classes = {
    "Not_Oriented": 0, 
    "Oriented": 1, 
    "Slanted": 2, 
    "Unidentified": 3, 
    "No_sample": 4,
}
transition_table = pd.DataFrame(
data=model.transmat_,
index=list(classes.keys()),
columns=list(classes.keys()))


## Output the transition matrix

In [59]:
pd.option_context('precision', 3)

In [60]:
transition_table

,Not_Oriented,Oriented,Slanted,Unidentified,No_sample
Not_Oriented,9.197867e-01,3.656876e-03,2.941763e-02,4.052531e-02,6.613449e-03
Oriented,7.301879e-03,8.521219e-01,1.372400e-01,2.251444e-03,1.084738e-03
Slanted,3.565343e-09,4.194937e-02,9.037174e-01,5.433323e-02,1.146448e-56
Unidentified,4.394421e-02,2.907449e-02,2.629513e-01,6.640300e-01,2.098000e-49
No_sample,8.333333e-02,1.204606e-243,6.187378e-72,1.058230e-54,9.166667e-01
